# WATERSHED SEGMENTATION ALGORITHM
Classic algorithm used for **segmentation** and is especially useful when extracting **touching or overlapping objects** in images

When utilizing the watershed algorithm we must start with **user-defined markers**

These markers can be either defined:
- manually point-and-click
- automatically or heuristically define them
    - thresholding operations
    - morphological operations
    
Based on these markers, the watershed algorithm treats pixels in our input image as **local elevation** (called a topography) — the method **“floods”** valleys, starting from the markers and moving outwards, until the valleys of different markers meet each other

- In order to obtain an accurate watershed segmentation, the markers must be correctly placed


## Segment and Extract objects 
- Import the libraries
- Load the image
- Apply Pyramid Mean Shift Filtering 
[PMSF]https://docs.opencv.org/2.4/modules/imgproc/doc/filtering.html#pyrmeanshiftfiltering
- Convert the image into grayscale
- Threshold the Mean Shifted image
- Detect Contours using findContours()
- Draw Contours using drawContours()
- Display the extracted contours

### Import the necessary libraries

### Load the image

### Apply Pyramid Mean Shift Filtering
cv2.PyrMeanShiftFiltering()
- src
    - The source 8-bit, 3-channel image
- dst
    - The destination image of the same format and the same size as the source
- sp
    - The spatial window radius
- sr
    - The color window radius
- maxLevel
    - Maximum level of the pyramid for the segmentation
- termcrit
    - Termination criteria: when to stop meanshift iterations

### Convert the image into Grayscale

### Threshold the image
- Use OTSU Simple Thresholding

### Detect Contours
**cv2.findContours()**
- image (recommeded to send its copy rather the original)
- type of contour
    - cv2.RETR_EXTERNAL
    - cv2.RETR_LIST
    - cv2.RETR_COMP
    - cv2.RETR_TREE
- approximation of contour
    - cv2.CHAIN_APPROX_SIMPLE
    - cv2.CHAIN_APPROX_NONE
- Returns a tuple
    - output image after applying contour detection
    - cnts list of contours detected
    - hierarchy of the contours

#### Helper function - grab_contours( )

In [10]:
def grab_contours(cnts): 
    if len(cnts) == 2: 
        cnts = cnts[0]
    elif len(cnts) == 3:
        cnts = cnts[1]
    else:
        raise Exception(("Contours tuple must have length 2 or "
                        "3, otherwise OpenCV changed their cv2.findContours " 
                        "return signature yet again. "
                        "Refer to OpenCV’s documentation in that case."))
    return cnts

### How many contours?

### Draw Contours
cv2.drawContours()
- image
- contours list
- contour index
    - -1 --> draw all of the contours
    - i --> draw single contour
- color of the contour line
    - Use green color
- thickness of the line

In [14]:
coins = image.copy()
for (i, c) in enumerate(cnts):
    # draw the contour
    ((x, y), _) = cv2.minEnclosingCircle(c)
    cv2.putText(image, "#{}".format(i + 1), (int(x) - 10, int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    

### Display the contours

In [15]:
cv2.imshow("Extraction of Coins", np.hstack([image, coins]))
cv2.waitKey(0)

13

### Problems???

In [16]:
# Uncomment the following to install scipy, scikit-image, imutils
# ! pip install scipy scikit-image imutils

## Watershed Segmentation

#### Usual Steps
- Load the image
- Apply Pyramid Mean Shift Filtering 
- Convert the image into grayscale
- Threshold the Mean Shifted image

#### New Steps
- Import new libraries
- Euclidean Distance Transform (EDT)
- Find Peaks in the Distance Map
- Perform a Connected Component Analysis on the local peaks using 8-connectivity
- Apply Watershed algorithm to get pixel labels
- Draw Contours by looping through pixel lables

#### Import new libraries

#### Euclidean Distance Transform (EDT)
Computes the Euclidean distance to the closest zero (background) pixel for each of the white (foreground) pixels and builds a distance map
![Distance Map](https://raw.githubusercontent.com/subashgandyer/datasets/main/images/Watershed-EDT-DistanceMap.png)

#### FInding peaks in the distance map

Peaks --> local maxima

Please note a minimum 20 pixel distance between each peak

#### Connected Component Analysis
[Wiki]https://en.wikipedia.org/wiki/Connected-component_labeling

Use **8-connectivity**

### Watershed Algorithm

Watershed assumes **markers** as **local minima** (valleys) in our distance map

Returns a matrix of **labels** for every pixel

Each pixel value as a unique label value

**Pixels that have the same label value belong to the same object**



#### How many Contours found?

#### Draw Contours
- Loop over the unique labels returned by watershed algorithm
- If it is not a background pixel
    - Allocate memory for the label region and draw it on the mask
    - Detect Contours in the mask and grab the largest one
    - Draw a circle enclosing the object
    - Place a number on the image
- If it is a background pixel
    - Ignore

In [27]:
for label in np.unique(labels):
    if label == 0:
        continue
        
    mask = np.zeros(gray.shape, dtype="uint8")
    mask[labels == label] = 255
    
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    
    ((x, y), r) = cv2.minEnclosingCircle(c)
    cv2.circle(image, (int(x), int(y)), int(r), (0, 255, 0), 2)
    
    cv2.putText(image, "#{}".format(label), (int(x) - 10, int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

#### Display the images

In [28]:
cv2.imshow("Watershed Extraction of coins", np.hstack([coins, image]))
cv2.waitKey(0)

13

#### Summary

Watershed algorithm is a classic segmentation algorithm used to detect and extract objects in images that are touching and/or overlapping

To apply the watershed algorithm:
- we need to define markers which correspond to the objects in our image
- These markers can be either user-defined or we can apply image processing techniques (such as thresholding) to find the **markers** for us
- When applying the watershed algorithm, it’s absolutely critical that we obtain accurate markers

Given our markers:
- we can compute the Euclidean Distance Transform
- and pass the distance map to the watershed function itself, which **“floods”** valleys in the distance map, starting from the initial markers and moving outwards
- Where the **“pools”** of water meet can be considered **boundary lines** in the segmentation process
- The output of the watershed algorithm is a set of **labels**, where each label corresponds to a unique object in the image
- loop over each of the labels individually and extract each object